# step1: use the exploreWifiLL.ipynb to generate the individual mobility data for each peoriod

In [2]:
import pandas as pd
import json
import urllib
import re
import datetime
import pytz
import matplotlib.pyplot as plt
import io
from datetime import datetime
import pytz
from pandas.io.json import json_normalize
import pandas as pd
import os
import csv
import operator
import itertools

In [ ]:
Folder_Path = r'E:\jupyter3\raw data'
os.chdir(Folder_Path)
with open ('2018-07-28_05','r', encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines()
    dataRawJson=[json.loads(data[i]) for i in range(len(data))]

In [ ]:
with open ('pri-2018-07-28_05','r',encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines()
dataFormatted = '['+re.sub('}\s*{','},{',data[0])+']' 
dataCleanJson=json.loads(dataFormatted)
dataCleanJsonLists=[[rec['data'][i].split(',')for i in range(len(rec['data']))]for rec in dataCleanJson]
validMacs = [[record[i][5] for i in range(len(record))] for record in dataCleanJsonLists]
uniqueValidMacs=set([mac for macList in validMacs for mac in macList])
numUniqueValidMacs = len(uniqueValidMacs)

In [ ]:
timeCorrection=8*60*60*1000
allTimes=[]
individuals={}
c=0
for record in dataRawJson:
    c+=1
    wifi=record['wifidata']
    apmac=wifi['apmac']
    ts=float(wifi['tssend'])+timeCorrection 
    allTimes.extend([ts])
    dateTime=datetime.datetime.utcfromtimestamp(ts/1000).strftime('%Y%m%d %H:%M:%S') 
    count=wifi['count']
    for obs in wifi['wifitalist']:
        valid=obs['mac'] in uniqueValidMacs
        newObs={'probe':apmac,
                'ts':ts,
                'datetime':dateTime,
                'rssi':obs['rssi']} 
        if obs['mac'] in individuals:
            individuals[obs['mac']]['observations'].append(newObs)
        else:
            individuals[obs['mac']]={'valid':valid,'observations':[newObs]}

# step2: covert the json format data into csv format 

In [ ]:
with open('indi072805.json', 'w') as outfile: 
    json.dump(individuals, outfile)

In [ ]:
Folder_Path = r'E:\jupyter3\raw data' #your own filepath
os.chdir(Folder_Path)
with open ('indi072805.json','r', encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines()
    IndividualJson=[json.loads(data[i]) for i in range(len(data))]
jsondata = IndividualJson[0]
key = list(IndividualJson[0].keys())
idx=0
result= json_normalize(IndividualJson[0][key[0]],'observations')
result.insert(loc=idx,column='id',value=key[0])
for i in range(1, len(key)):
    test = json_normalize(IndividualJson[0][key[i]],'observations')
    test.insert(loc=idx,column='id',value=key[i])
    result = result.append(test,ignore_index=True)
#print(result)
result.to_csv('indi072805.csv',sep=',',encoding='utf-8')

## merge the six csvs into one, which is the data for one whole day

In [3]:
Folder_Path = r'E:\jupyter3\raw data\0728_csv'
SaveFile_Path =  r'E:\jupyter3\raw data\0728_csv'
SaveFile_Name = r'allindi_0728.csv'
os.chdir(Folder_Path)
file_list = os.listdir()
df = pd.read_csv(Folder_Path +'\\'+ file_list[0]) 
df.to_csv(SaveFile_Path+'\\'+ SaveFile_Name,encoding="utf_8_sig",index=False)
for i in range(1,len(file_list)):
    df = pd.read_csv(Folder_Path + '\\'+ file_list[i])
    df.to_csv(SaveFile_Path+'\\'+ SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+')

# step3: parse the data and add xy coordinates to the probes; thus we can calculate the coordinates for each individual at a certain time

In [ ]:
Folder_Path = r'E:\jupyter3\raw data\csv'
os.chdir(Folder_Path)
indi0728 = pd.read_csv('all_indi0728.csv')

In [ ]:
apmac_info = pd.read_csv('apmac_info.csv')

In [ ]:
## apmac_info is the records of the xy coordinates for each probe

In [ ]:
indi_final = indi0728.merge(apmac_info, on="probe", how = 'inner').fillna(0)
indi_final.sort_values('id')

In [ ]:
#scripts for how to calculate the coordinates for each individual will be added later; I conducted it via Stata

## Parse the data to the format of list for individuals

In [ ]:
## note:information for each individual: {'id': agent_id, 'x': list_of_x_coordinates, 'y': list_of_y_coordinates, 'timestamps': list_of_timestamps }
then list for all the individuals:[agent1_dict,  agent2_dict, ....]

In [ ]:
ind = pd.read_csv('0728id_xy_version2.csv')
with open('0728id_xy_version2.csv') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

with open('0728id_xy_version2.json', 'w') as f:
    json.dump(rows, f)

In [ ]:
with open ('0728id_xy_version2.json','r', encoding='utf-8',errors='replace') as myfile:
    data = myfile.readlines()
    ind_data=[json.loads(data[i]) for i in range(len(data))]

In [ ]:
with open('0728id_xy_version2.json') as f:
    things = [json.loads(line) for line in f]

In [ ]:
new_things = {}
for thing in things:
    thing_id = thing[0]['id']
    try:
        old_thing = new_things[thing_id]
    except KeyError:
        new_things[thing_id] = thing
    else:
        old_thing['x'].extend(thing['x'])
        old_thing['y'].extend(thing['y'])
        old_thing['timestamps'].extend(thing['timestamps'])
new_things = new_things.values()

In [ ]:
def merge(things):
    return {'id': things[0]['id'],
            'x': list(itertools.chain(float(t['x']) for t in things)),
            'y': list(itertools.chain(float(t['y']) for t in things)),
            'timestamps': list(itertools.chain(float(t['timestamps']) for t in things))}
sorted_things = sorted(things[0], key=operator.itemgetter('id'))
grouped_things = itertools.groupby(sorted_things, key=operator.itemgetter('id'))
agents = [merge(list(group)) for key, group in grouped_things]

In [ ]:
import numpy as np

def maxDistFromGroup(newPoint, pointGroup):
    #takes a point and an arra of points and returns 
    #the max dist of any point in the group from this new point
    maxDist=0
    for p in pointGroup:
        distN=np.sqrt( (p[1] - newPoint[1])**2 + (p[0] - newPoint[0])**2)
        maxDist=max(distN,maxDist)
    return maxDist

In [ ]:
maxRoam=15
minStay=20
def extractStays(agents, maxRoam, minStay):
    #takes a list of agent objects, and returns a list of stays for each agent
    #each agent in the input data should be a dictionary with an id, a list of x coordinates, a list y coordinates and a list of timestamps
    # the lists of coordintes and timestamps must also be sorted from earliest to latest
    #maxRoam is the distance threshold and minStay is the time threshold
    stayData=[]
    for a in range(len(agents)):
        xList, yList, timeList =agents[a]['x'], agents[a]['y'], agents[a]['timestamps']
        stays=[]
        i=0
        while i<len(xList):
            j=i
            while (j+1)<len(xList) and maxDistFromGroup([xList[j+1],yList[j+1]], [[xList[ij],yList[ij]] for ij in range(i,j+1)])<maxRoam:
                j+=1
            #create a stay from the middle position with arrival and departure time
            jj=min(j+1,len(xList)-1)
            # departure time is the timestamp of the next obs after this cluster. Unless this cluster contains the last point in the series
            if timeList[jj]-timeList[i]>(minStay):
                stays.extend([{'p':[xList[int((i+j)/2)],yList[int((i+j)/2)]], 'start':timeList[i],'end':timeList[jj], 'n':(j-i+1)}])       
            i=j+1
        if stays: # if stays is empty, no need to add anything
            #add the new agent to the stay data
            stayData.append({'stays':stays, 'id': agents[a]['id']})
    return stayData
#here we set the maxRoam is 15 meters and the minStay is 20 minutes
if __name__ == '__main__':
    maxRoam=15000
    minStay=20*60*1000
    result = extractStays(agents, maxRoam, minStay)
    print (result[0])
#result is the stay point extracted